In [ ]:
#langchain
from openai import OpenAI

In [ ]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

# Test the api_key


In [ ]:
responsed = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"}]

)
responsed.choices[0].message.content

'The Los Angeles Dodgers won the World Series in 2020 by defeating the Tampa Bay Rays.'

#Scrapper

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

def get_text_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove scripts, styles, and navs
        for tag in soup(["script", "style", "nav", "footer", "header"]):
            tag.decompose()

        text = soup.get_text(separator=' ', strip=True)
        return text
    except Exception as e:
        print(f"Error extracting {url}: {str(e)}")
        return ""

def crawl_website(base_url, max_pages=5):
    visited = set()
    to_visit = [base_url]
    content = ""

    while to_visit and len(visited) < max_pages:
        current = to_visit.pop(0)
        if current in visited:
            continue
        visited.add(current)
        print(f"Crawling: {current}")
        page_text = get_text_from_url(current)
        content += f"\n\n--- Content from: {current} ---\n\n{page_text}"

        try:
            response = requests.get(current, timeout=10)
            soup = BeautifulSoup(response.text, 'html.parser')
            for link in soup.find_all("a", href=True):
                href = link["href"]
                full_url = urljoin(base_url, href)
                if base_url in full_url and full_url not in visited:
                    to_visit.append(full_url)
        except:
            continue

    return content


#Save Text (To avoid constant scrape)

In [ ]:
if __name__ == "__main__":
    website = "https://diamondadverts.com"  # Replace with the client site
    extracted_content = crawl_website(website, max_pages=10)

    with open("site_content.txt", "w", encoding="utf-8") as f:
        f.write(extracted_content)

    print("Website content saved to site_content.txt")


Crawling: https://diamondadverts.com
Crawling: https://diamondadverts.com#content
Crawling: https://diamondadverts.com/
Crawling: https://diamondadverts.com/about-us/
Crawling: https://diamondadverts.com/social-media-management/
Crawling: https://diamondadverts.com/website-design/
Crawling: https://diamondadverts.com/content-creation-and-branding/
Crawling: https://diamondadverts.com/blogs/
Crawling: https://diamondadverts.com/contact-us/
Crawling: https://diamondadverts.com/2025-seo-hack-strategic-backlinking-that-works/
Website content saved to site_content.txt


#Functionality Define
For proper implementation

In [ ]:
SUPPORT_EMAIL = "inof@diamondadverts.com"
TOOLS = "when asked about 'James' reply he is unavailable between 1st of January and 2nd of may"

#Deploy

In [ ]:
from flask import Flask, request, jsonify
import openai

app = Flask(__name__)

# Load site content
with open("/content/site_content.txt", "r", encoding="utf-8") as f:
    WEBSITE_CONTENT = f.read()

WEBSITE_URL = "https://diamondadverts.com"  # Replace with your site

SYSTEM_PROMPT = f"""
You are a helpful and professional assistant for the website at {WEBSITE_URL}.
Only answer questions using this website content:

{WEBSITE_CONTENT} and {TOOLS}

When asked something outside the scope of {WEBSITE_CONTENT}, check {TOOLS} to answer

If a question is outside the scope of the site, politely say you can't answer it.
Always end with a suggestion to visit more aboout the website.
When asked to speak to a human direct them to  {SUPPORT_EMAIL}
"""

@app.route("/chat", methods=["POST"])
def chat():
    data = request.json
    user_message = data.get("message")

    if not user_message:
        return jsonify({"error": "No message provided."}), 400

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_message}
            ]
        )
        reply = response.choices[0].message.content.strip()
        return jsonify({"reply": reply})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# if __name__ == "__main__":
#     app.run(debug=True)


In [ ]:
#gradio test

In [ ]:
import gradio as gr
def gradio_chat(user_message):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_message}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {str(e)}"

demo = gr.Interface(
    fn=gradio_chat,
    inputs=gr.Textbox(lines=2, placeholder="Ask something about the website..."),
    outputs="text",
    title="Diamond Adverts Chatbot",
    description="Ask anything about the Diamond Adverts website"
)

demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dcb5f2fd3cb4ba1679.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://dcb5f2fd3cb4ba1679.gradio.live


In [ ]:
#Using_ollama


In [5]:
import requests

OLLAMA_URL = "https://43b6-62-49-23-111.ngrok-free.app"

payload = {
    "model": "llama3.2",
    "prompt": "Tell me a fun fact about the moon.",
}

response = requests.post(f"{OLLAMA_URL}/api/generate", json=payload)

# Print the raw response text to debug
print("Raw Response Text:")
print(response.text)


Raw Response Text:


Error: Could not decode JSON from the response.
Please check the server logs for the Ollama instance.
